# Transform Laws
---
Essa função padroniza a chamada de leis, de forma que: Lei nº 99.999.999 seja substituída por **ST_LEI_99999999**.
# Recursos externos
--- 
A função não usa nenhum arquivo externo. Entretanto, as seguintes variáveis e funções são externas/auxiliares (necessitam importação):
- ST_PREFIX => "ST_"
- import re

In [1]:
import re

ST_PREFIX = "ST_"

def _transform_legislations(
    text,
    prefix_result,
    re_legislation,
    is_multiple=False,
    identification=None,
    previous_token=None,
    separator=None,
):
    "Create custom tokens for legislation"

    def build_token(m):
        if m.group("ignored_expression"):
            result = m.group("ignored_expression")
        else:
            result = ""
        result = (
            result
            + ST_PREFIX
            + prefix_result
            + m.group("identification")
            .replace('"', "")
            .replace("“", "")
            .replace("”", "")
            .replace(".", "")
            .replace(",", "")
            .replace("º", "")
            .lstrip("0")
            .upper()
        )
        return result

    def build_token_multi(m):
        result = (
            m.group("ignored_expression")
            + ST_PREFIX
            + prefix_result
            + m.group("identification")
            .replace('"', "")
            .replace("“", "")
            .replace("”", "")
            .replace(".", "")
            .replace(",", "")
            .replace("º", "")
            .lstrip("0")
            .upper()
        )
        return result

    # replaces first occurrences
    text = re_legislation.sub(build_token, text)

    # replaces multiple occurrences, if exists
    if is_multiple:
        regex_legislation_multi = re.compile(
            fr"(?:(?P<ignored_expression>{previous_token}{separator}){identification})",
            flags=re.I,
        )
        while regex_legislation_multi.findall(text) != []:
            text = regex_legislation_multi.sub(build_token_multi, text)

    return text


## transform_laws
---

In [2]:
def transform_laws(text, **kwargs):
    #Create custom tokens for laws

    law_types = (
        "complementar(?:es)?",
        "federa(?:l|is)",
        "estadua(?:l|is)",
        "municipa(?:l|is)",
    )
    law_types = "|".join(law_types)

    word_number = r"(?:n[oº]?\.?|n\.?º|n[úu]mero)"
    number = r"(?:\d+(?:[\.,]\d+)*)"
    year = fr"(?:[/-]{number})?"
    prefix_result = "LEI_"
    previous_token = fr"{ST_PREFIX}{prefix_result}{number}"
    separator = r"\s*[,e]\s+"
    law = fr"(?P<identification>{number}){year}"
    re_legislation = re.compile(
        fr"(?P<ignored_expression>)(?<!\w)(leis?|l\.)(\s+({law_types}))?\s+(?:{word_number}\s*)?{law}",
        flags=re.I,
    )

    return _transform_legislations(
        text,
        prefix_result,
        re_legislation,
        True,
        law,
        previous_token,
        separator,
    )


### Procedimentos e Regras
---
É necessario utilizar função **_transform_legislations** como auxiliar.

### Exemplos de Funcionamento:
---
Entradas que esperam a transformação da função e suas saídas:

1. **Entrada:** Lei 4.504/64 / **Saída:** ST_LEI_4504
2. **Entrada:** Lei nº 99.999.999/2100 / **Saída:** ST_LEI_99999999
3. **Entrada:** Lei nº 7.844, de 1989 / **Saída:** SST_LEI_7844, de 1989
4. **Entrada:** Informação - Lei nº 5.250-1967 / **Saída:** Informação - ST_LEI_5250
5. **Entrada:** Lei Complementar nº 70/91 / **Saída:** ST_LEI_70
6. **Entrada:** Lei Municipal 67478 / **Saída:** ST_LEI_67478
7. **Entrada:** Lei Federal 67478 / **Saída:** ST_LEI_67478
8. **Entrada:** Lei Estadual 67478 / **Saída:** ST_LEI_67478
9. **Entrada:** Lei Complementar 67478 / **Saída:** ST_LEI_67478
10. **Entrada:** Lei Antiterrorismo - Lei 13.260 / **Saída:** Lei Antiterrorismo - ST_LEI_13260
11. **Entrada:** Lei 3.4 tribunal justiça  estado bahia poder judiciário salvador juizado cível defesa / **Saída:**  ST_LEI_34 tribunal justiça  estado bahia poder judiciário salvador juizado cível defesa


In [5]:
entradas = """
  Lei 4.504/64
  Lei nº 99.999.999/2100
  Lei nº 7.844, de 1989
  Informação - Lei nº 5.250-1967
  Lei Complementar nº 70/91
  Lei Municipal 67478
  Lei Federal 67478
  Lei Estadual 67478
  Lei Complementar 67478
  Lei Antiterrorismo - Lei 13.260
  Lei 3.4 tribunal justiça  estado bahia poder judiciário salvador juizado cível defesa
"""

print("Entrada: ", entradas)

Entrada:  
  Lei 4.504/64
  Lei nº 99.999.999/2100
  Lei nº 7.844, de 1989
  Informação - Lei nº 5.250-1967
  Lei Complementar nº 70/91
  Lei Municipal 67478
  Lei Federal 67478
  Lei Estadual 67478
  Lei Complementar 67478
  Lei Antiterrorismo - Lei 13.260
  Lei 3.4 tribunal justiça  estado bahia poder judiciário salvador juizado cível defesa



In [6]:
print("Saida: ",transform_laws(entradas))

Saida:  
  ST_LEI_4504
  ST_LEI_99999999
  ST_LEI_7844, de 1989
  Informação - ST_LEI_5250
  ST_LEI_70
  ST_LEI_67478
  ST_LEI_67478
  ST_LEI_67478
  ST_LEI_67478
  Lei Antiterrorismo - ST_LEI_13260
  ST_LEI_34 tribunal justiça  estado bahia poder judiciário salvador juizado cível defesa



# Aplicação no dataset Victor Small
---
### Tempo gasto para rodar todo o dataset Victor e fazer possíveis alterações: 
---
*35s*

In [7]:
import pandas as pd

test = pd.read_csv('./data/train_small.csv')
text = test['body']

In [8]:
print("Dados de entrada:\n",text)

Dados de entrada:
 0         {"tribunal justiça estado bahia poder judiciár...
1         {"excelentíssimo senhor doutor juiz direito ju...
2         {"razões recurso inominado recorrente atlantic...
3         {"empresa recorrente tornou credora dos débito...
4         {"entretanto verdade parte apelante tornou tit...
                                ...                        
149212    {"supremo tribunal federal recurso extraordiná...
149213    {"seção recursos extraordinários mandado intim...
149214    {"ttar qsvòwi a edewrr seção recursos extraord...
149215    {"ertidao certifico dou que dirigi setor autar...
149216    {"supremo tribunal federal secretaria judiciár...
Name: body, Length: 149217, dtype: object


In [10]:
%%time
index = 0
for item in text:
  text.at[index] = transform_laws(item)
  index += 1

CPU times: user 34.5 s, sys: 396 ms, total: 34.8 s
Wall time: 34.4 s


In [11]:
print("Dados de saida: \n", text)

Dados de saida: 
 0         {"tribunal justiça estado bahia poder judiciár...
1         {"excelentíssimo senhor doutor juiz direito ju...
2         {"razões recurso inominado recorrente atlantic...
3         {"empresa recorrente tornou credora dos débito...
4         {"entretanto verdade parte apelante tornou tit...
                                ...                        
149212    {"supremo tribunal federal recurso extraordiná...
149213    {"seção recursos extraordinários mandado intim...
149214    {"ttar qsvòwi a edewrr seção recursos extraord...
149215    {"ertidao certifico dou que dirigi setor autar...
149216    {"supremo tribunal federal secretaria judiciár...
Name: body, Length: 149217, dtype: object


### Configurações da máquina em que a execução foi feita:
---
* **Processador:** Intel(R) Core(TM) i5
* **Memória RAM:** 4GB
